In [1]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers
from keras.callbacks import TensorBoard
import datetime

import numpy as np

Using TensorFlow backend.


In [2]:
def get_timestamp():
    return '{:%Y-%m-%d-%H-%M-%S}'.format(datetime.datetime.now())

In [3]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [4]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(cifar10)

In [5]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(x_train)



In [7]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=x_train.shape[1:]))
small_cnn.add(Conv2D(32, (3, 3), padding = "same", activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(64, (3, 3), padding = "same", activation='relu'))
small_cnn.add(Conv2D(64, (3, 3), padding = "same", activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Conv2D(128, (3, 3), padding = "same", activation='relu'))
small_cnn.add(Conv2D(128, (3, 3), padding = "same", activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.3))

small_cnn.add(Flatten())

small_cnn.add(Dense(units=128, activation='relu'))
small_cnn.add(Dropout(0.3))
small_cnn.add(Dense(units=128, activation='relu'))
small_cnn.add(Dropout(0.3))

small_cnn.add(Dense(units=10, activation='softmax'))

In [8]:
#model = linear
model = small_cnn

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [8]:
#without data augmentation, without last dropouts
n_train    = 6000
batch_size = 128
epochs     = 60

name = 'graph'

tensorboard = TensorBoard( log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:1000], y_test[:1000]),
         callbacks=[tensorboard])

Train on 6000 samples, validate on 1000 samples
Epoch 1/60
6000/6000 [==============================] - 47s - loss: 2.2859 - acc: 0.1255 - val_loss: 2.2299 - val_acc: 0.1690
Epoch 2/60
6000/6000 [==============================] - 54s - loss: 2.2163 - acc: 0.1837 - val_loss: 2.1268 - val_acc: 0.2290
Epoch 3/60
6000/6000 [==============================] - 53s - loss: 2.0966 - acc: 0.2367 - val_loss: 2.0480 - val_acc: 0.2800
Epoch 4/60
6000/6000 [==============================] - 50s - loss: 2.0265 - acc: 0.2660 - val_loss: 1.9932 - val_acc: 0.3050
Epoch 5/60
6000/6000 [==============================] - 46s - loss: 1.9828 - acc: 0.2833 - val_loss: 1.8664 - val_acc: 0.3330
Epoch 6/60
6000/6000 [==============================] - 48s - loss: 1.8602 - acc: 0.3267 - val_loss: 1.7886 - val_acc: 0.3400
Epoch 7/60
6000/6000 [==============================] - 52s - loss: 1.8013 - acc: 0.3365 - val_loss: 1.7377 - val_acc: 0.3690
Epoch 8/60
6000/6000 [==============================] - 56s - loss: 1.

KeyboardInterrupt: 

In [ ]:
_, acc = model.evaluate(x_test[:1000], y_test[:1000], verbose=0)
print('Test accuracy:', acc)

In [9]:
#with data augmentation, without last dropouts
n_train    = 6000
batch_size = 128
epochs     = 60

name = 'graph'

tensorboard = TensorBoard( log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit_generator(datagen.flow(x_train[:6000], y_train[:6000], batch_size=batch_size),
          steps_per_epoch = n_train / batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:1000], y_test[:1000]),
          callbacks=[tensorboard])

Epoch 1/60
47/46 [==============================] - 32s - loss: 2.2883 - acc: 0.1184 - val_loss: 2.2299 - val_acc: 0.2230
Epoch 2/60
47/46 [==============================] - 36s - loss: 2.1850 - acc: 0.1900 - val_loss: 2.0865 - val_acc: 0.2310
Epoch 3/60
47/46 [==============================] - 36s - loss: 2.1003 - acc: 0.2225 - val_loss: 2.0021 - val_acc: 0.2690
Epoch 4/60
47/46 [==============================] - 35s - loss: 2.0693 - acc: 0.2326 - val_loss: 1.9956 - val_acc: 0.2650
Epoch 5/60
47/46 [==============================] - 36s - loss: 2.0167 - acc: 0.2543 - val_loss: 2.0358 - val_acc: 0.2420
Epoch 6/60
47/46 [==============================] - 35s - loss: 1.9987 - acc: 0.2557 - val_loss: 1.9641 - val_acc: 0.2500
Epoch 7/60
47/46 [==============================] - 38s - loss: 1.9430 - acc: 0.2824 - val_loss: 1.9906 - val_acc: 0.2830
Epoch 8/60
47/46 [==============================] - 46s - loss: 1.9178 - acc: 0.2903 - val_loss: 1.7804 - val_acc: 0.3580
Epoch 9/60
47/46 [======

In [11]:
_, acc = model.evaluate(x_test[:1000], y_test[:1000], verbose=0)
print('Test accuracy:', acc)

Test accuracy: 0.606


In [ ]:
#with data augmentation, with last dropouts
n_train    = 6000
batch_size = 128
epochs     = 60

name = 'graph'

tensorboard = TensorBoard( log_dir='./{0}/{1}'.format(name, get_timestamp()), histogram_freq=0,
                          write_graph=True, write_images=False)

model.fit_generator(datagen.flow(x_train[:6000], y_train[:6000], batch_size=batch_size),
          steps_per_epoch = n_train / batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:1000], y_test[:1000]),
          callbacks=[tensorboard])

Epoch 1/60
47/46 [==============================] - 51s - loss: 2.2974 - acc: 0.1136 - val_loss: 2.2869 - val_acc: 0.1380
Epoch 2/60
47/46 [==============================] - 46s - loss: 2.2339 - acc: 0.1670 - val_loss: 2.1340 - val_acc: 0.2620
Epoch 3/60
47/46 [==============================] - 43s - loss: 2.1265 - acc: 0.2007 - val_loss: 2.0262 - val_acc: 0.2650
Epoch 4/60
47/46 [==============================] - 42s - loss: 2.0885 - acc: 0.2152 - val_loss: 2.0280 - val_acc: 0.2710
Epoch 5/60
47/46 [==============================] - 42s - loss: 2.0440 - acc: 0.2375 - val_loss: 1.9359 - val_acc: 0.2950
Epoch 6/60
47/46 [==============================] - 41s - loss: 2.0277 - acc: 0.2393 - val_loss: 1.8915 - val_acc: 0.3230
Epoch 7/60
47/46 [==============================] - 41s - loss: 1.9855 - acc: 0.2570 - val_loss: 1.8625 - val_acc: 0.3180
Epoch 8/60
47/46 [==============================] - 39s - loss: 1.9528 - acc: 0.2709 - val_loss: 1.8636 - val_acc: 0.3080
Epoch 9/60
47/46 [======